In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind

In [2]:
prot= pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD Shared folder\Datasets\Set C1\CSF extracellular original.xlsx", 
                    header=0, index_col="Gene names")
prot 

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,ALS 11,ALS 12,Control 1,Control 2,Control 3,Control 4,Control 5
Gene names,,,,,,,,,,,,,,,,,
SERPINB3,14.872378,12.859114,11.854119,14.273470,14.716500,10.948828,13.878442,13.416773,14.861255,14.406942,16.500438,11.970441,15.340614,13.751069,16.501273,14.961888,16.815706
UBA1,9.583395,7.310815,7.484620,9.411431,11.072066,6.889876,11.973624,10.157290,10.017449,7.420336,9.651234,NaN,5.365793,6.531231,13.012980,10.606512,7.025345
SERPINB4,14.265418,11.702133,10.919117,12.899090,13.673972,9.552004,12.827312,12.930742,13.720898,13.375603,15.179760,10.588010,14.056433,12.964267,15.447305,13.737367,15.744283
THBD,10.095219,8.356654,10.226136,11.788199,14.434087,9.091466,13.212333,11.546541,11.889096,10.724110,9.126712,8.118944,11.412722,9.236970,14.217546,13.152418,6.686906
ILK,15.422415,12.103125,12.978554,16.006535,15.319039,10.233732,13.901977,14.907776,16.199154,14.326627,16.129916,10.740811,14.098989,11.162352,14.576346,12.906595,11.095222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ST13P4,13.509425,12.660875,8.781761,13.536178,16.852569,7.678874,16.229784,16.209696,14.128000,14.386630,14.434512,6.374744,9.624624,12.439679,17.370361,13.822800,9.136051
OAF,12.379922,10.075540,9.222113,11.661151,16.691011,8.456512,14.640553,13.264945,13.576966,11.452221,12.938884,8.719383,12.747210,10.701577,12.916629,12.482860,13.196174
PP6455,12.656166,11.037789,10.129555,13.272658,15.231528,7.815388,14.338329,13.892981,13.399063,12.473510,13.944493,9.468768,13.359777,10.955733,12.854068,13.472045,12.568817


In [3]:
prot.shape


(1018, 17)

In [4]:
# Columns belonging to Group 1 and 2
ALS_columns = [col for col in prot.columns if "ALS" in col]
Control_columns = [col for col in prot.columns if "Control" in col]


# Filter rows where at least 50% of Group 1 values are present, do the same for Group 2
filtered_als = prot[prot[ALS_columns].notna().mean(axis=1) >= 0.5]
cont_filt = filtered_als[filtered_als[Control_columns].notna().mean(axis=1) >= 0.5]

In [5]:
cont_filt.shape

(1011, 17)

In [6]:
cont_filt.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,ALS 11,ALS 12,Control 1,Control 2,Control 3,Control 4,Control 5
Gene names,,,,,,,,,,,,,,,,,
SERPINB3,14.872378,12.859114,11.854119,14.273470,14.716500,10.948828,13.878442,13.416773,14.861255,14.406942,16.500438,11.970441,15.340614,13.751069,16.501273,14.961888,16.815706
UBA1,9.583395,7.310815,7.484620,9.411431,11.072066,6.889876,11.973624,10.157290,10.017449,7.420336,9.651234,NaN,5.365793,6.531231,13.012980,10.606512,7.025345
SERPINB4,14.265418,11.702133,10.919117,12.899090,13.673972,9.552004,12.827312,12.930742,13.720898,13.375603,15.179760,10.588010,14.056433,12.964267,15.447305,13.737367,15.744283
THBD,10.095219,8.356654,10.226136,11.788199,14.434087,9.091466,13.212333,11.546541,11.889096,10.724110,9.126712,8.118944,11.412722,9.236970,14.217546,13.152418,6.686906
ILK,15.422415,12.103125,12.978554,16.006535,15.319039,10.233732,13.901977,14.907776,16.199154,14.326627,16.129916,10.740811,14.098989,11.162352,14.576346,12.906595,11.095222


In [1]:
# Automatically identify group columns by their prefixes
als = [col for col in cont_filt.columns if col.startswith('ALS ')]
control = [col for col in cont_filt.columns if col.startswith('Control ')]

# Perform t-tests row-wise
results = []
for i, row in cont_filt.iterrows():
    t_stat, p_value = ttest_ind(row[als].dropna(), row[control].dropna())
    results.append({'Gene names': i, 't_stat': t_stat, 'p_value': p_value})
    
# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

NameError: name 'cont_filt' is not defined

In [8]:
#prot.index[prot.index.duplicated()]

Index([], dtype='object', name='Gene names')

In [9]:
#len(prot.index.unique()), len(prot.index)

(1018, 1018)

In [10]:
#len(results_df.set_index('Gene names').index.unique()), len(results_df.set_index('Gene names').index)

(1011, 1011)

In [11]:
# Add the results DataFrame to the original data DataFrame
prot_stat = pd.concat([prot, results_df.set_index('Gene names')], axis=1)
prot_stat.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,ALS 11,ALS 12,Control 1,Control 2,Control 3,Control 4,Control 5,t_stat,p_value
Gene names,,,,,,,,,,,,,,,,,,,
SERPINB3,14.872378,12.859114,11.854119,14.273470,14.716500,10.948828,13.878442,13.416773,14.861255,14.406942,16.500438,11.970441,15.340614,13.751069,16.501273,14.961888,16.815706,-2.219736,0.042271
UBA1,9.583395,7.310815,7.484620,9.411431,11.072066,6.889876,11.973624,10.157290,10.017449,7.420336,9.651234,NaN,5.365793,6.531231,13.012980,10.606512,7.025345,0.560852,0.583764
SERPINB4,14.265418,11.702133,10.919117,12.899090,13.673972,9.552004,12.827312,12.930742,13.720898,13.375603,15.179760,10.588010,14.056433,12.964267,15.447305,13.737367,15.744283,-2.151793,0.048111
THBD,10.095219,8.356654,10.226136,11.788199,14.434087,9.091466,13.212333,11.546541,11.889096,10.724110,9.126712,8.118944,11.412722,9.236970,14.217546,13.152418,6.686906,-0.184160,0.856354
ILK,15.422415,12.103125,12.978554,16.006535,15.319039,10.233732,13.901977,14.907776,16.199154,14.326627,16.129916,10.740811,14.098989,11.162352,14.576346,12.906595,11.095222,1.198795,0.249204


In [12]:
#df['average'] = df.mean(numeric_only=True, axis=1)
prot_stat['Mean Control'] = prot_stat.loc[:, prot_stat.columns.str.startswith('Control ')].mean(axis=1)
prot_stat['Mean ALS'] = prot_stat.loc[:, prot_stat.columns.str.startswith('ALS ')].mean(axis=1)

In [13]:
#replace the NA values with the mean                                
#prot.loc[:, prot.columns.str.startswith('Control ')] = prot.loc[:, prot.columns.str.startswith('Control ')].T.fillna(prot['Mean Control']).T
#prot.loc[:, prot.columns.str.startswith('ALS ')] = prot.loc[:, prot.columns.str.startswith('ALS ')].T.fillna(prot['Mean ALS']).T

In [14]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(prot_stat['p_value'], method='fdr_bh')
prot_stat['p_adjusted'] = p_adjusted
prot_stat.head()


,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,Control 1,Control 2,Control 3,Control 4,Control 5,t_stat,p_value,Mean Control,Mean ALS,p_adjusted
Gene names,,,,,,,,,,,,,,,,,,,,,
SERPINB3,14.872378,12.859114,11.854119,14.273470,14.716500,10.948828,13.878442,13.416773,14.861255,14.406942,...,15.340614,13.751069,16.501273,14.961888,16.815706,-2.219736,0.042271,15.474110,13.713225,NaN
UBA1,9.583395,7.310815,7.484620,9.411431,11.072066,6.889876,11.973624,10.157290,10.017449,7.420336,...,5.365793,6.531231,13.012980,10.606512,7.025345,0.560852,0.583764,8.508372,9.179285,NaN
SERPINB4,14.265418,11.702133,10.919117,12.899090,13.673972,9.552004,12.827312,12.930742,13.720898,13.375603,...,14.056433,12.964267,15.447305,13.737367,15.744283,-2.151793,0.048111,14.389931,12.636172,NaN
THBD,10.095219,8.356654,10.226136,11.788199,14.434087,9.091466,13.212333,11.546541,11.889096,10.724110,...,11.412722,9.236970,14.217546,13.152418,6.686906,-0.184160,0.856354,10.941312,10.717458,NaN
ILK,15.422415,12.103125,12.978554,16.006535,15.319039,10.233732,13.901977,14.907776,16.199154,14.326627,...,14.098989,11.162352,14.576346,12.906595,11.095222,1.198795,0.249204,12.767901,14.022472,NaN


In [15]:
#Fold Change
prot_stat["FC ALS-C"]= prot_stat["Mean ALS"]/prot_stat["Mean Control"]

In [16]:
#Log 2 FC
prot_stat["Log2FC ALS-C"]=np.log2(prot_stat["FC ALS-C"])

In [18]:
prot_stat.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\CSF extracellular\CSF extracellular short stats.xlsx")